In [ ]:
connection=mysql.connector.connect(database="p",host="localhost",user='root')
mycursor = connection.cursor()
ls=["نطق وتخاطب","ممارسة عامة","معامل تحاليل","مسالك بولية","مراكز أشعة","كلى","كبد","عيون","علاج طبيعي واصابات ملاعب","علاج الآلام","طب تقويمي","طب المسنين","طب الأسرة","صدر وجهاز تنفسي","سمعيات","سكر وغدد صماء","روماتيزم","ذكورة وعقم","حقن مجهري واطفال انابيب","حساسية ومناعة","جهاز هضمي ومناظير","جراحة مخ واعصاب","جراحة قلب وصدر","جراحة عمود فقري","جراحة عامة","جراحة سمنة ومناظير","جراحة تجميل","جراحة اوعية دموية","جراحة أورام","جراحة اطفال","تخسيس وتغذية","باطنة","اورام","امراض دم","قلب واوعية دموية","انف واذن وحنجرة","نساء وتوليد","عظام","مخ واعصاب","اطفال وحديثي الولادة","نفسي","اسنان","جلدية"]
ls.reverse()


response=requests.get('https://www.vezeeta.com/ar/specialties')
soup=BeautifulSoup(response.text,'html.parser')

url_original="https://www.vezeeta.com"
#التخصص 
id_general_specialty=0
id_doctor=0
id_con=0
count_g=0
id_sub_apichalist=0
for i in soup.find_all("div",attrs={"class":"Categoriesstyle__CollapsibleWrapper-sc-ftqhej-11 csguGo"}):
    #print(ls[count_g])
    id_general_specialty=id_general_specialty+1
    
    name_id_general_specialty=ls[count_g]
    count_g=count_g+1
    
    print(name_id_general_specialty)
    sql = "INSERT INTO general_specialty (id,type) VALUES (%s, %s)"
    val = (id_general_specialty,name_id_general_specialty)
    mycursor.execute(sql, val)

    connection.commit()
    
    
    for i2 in i.find_all("div",attrs={"class":"Categoriesstyle__SlideAnimation-sc-ftqhej-0 eVUZAC"}):
        #print(i2.find("a").get("title"))
        name_sub_apichalis=i2.find("a").get("title")
        id_sub_apichalist=id_sub_apichalist+1
        #print(id_sub_apichalist)
 
        sql = "INSERT INTO subspecialty (id,id_general_specialty,types_subspecialty) VALUES (%s, %s,%s)"
        val = (id_sub_apichalist,id_general_specialty,name_sub_apichalis)
        mycursor.execute(sql, val)

        connection.commit()
        
        
        # الدخول في التخصص
        url_original="https://www.vezeeta.com"
        url2=url_original+i2.find("a").get("href")
        #print(url2)
        response2=requests.get(url2)
        soup2=BeautifulSoup(response2.text,'html.parser')
        x=1   
        # الدخول في لينكات next
        while(soup2.find("a",attrs={"class":"Paginationstyle__TransparentA-sc-4txdoy-7 fKZvmm"})):
            # 'طباعة الاسماء'
            for i3 in soup2.find_all("div",attrs={"class":"CommonStylesstyle__ColDirection-sc-1vkcu2o-1 dFggjY"}):
                #--------------name---------------------
                
                first=i3.find("span",attrs={"class":"DoctorCardSubComponentsstyle__Text-sc-1vq3h7c-14 DoctorCardSubComponentsstyle__TitleText-sc-1vq3h7c-16 qnXIt"}).get_text()
                second=i3.find("span",attrs={"class":"DoctorCardSubComponentsstyle__Text-sc-1vq3h7c-14 DoctorCardSubComponentsstyle__DoctorNameText-sc-1vq3h7c-15 fpPjgt"}).get_text()
                name=first+" "+second
                #print(name)
                #---------name-----------
                
                #-----------id----------
                id_doctor=id_doctor+1
                #print(id_doctor)
                #-------id----------
                
                #------address-----
                address=i3.find("span",attrs={"class":"DoctorCardstyle__Text-sc-uptab2-4 gsYoeb"}).get_text()
                #----------------address------------------
                #-------------price-------------
                if i3.find("span",attrs={"itemprop":"priceRange"}):
                    price=i3.find("span",attrs={"itemprop":"priceRange"}).get_text().strip()
                else:
                    continue
                #print(price)
                #-----------price---------------------
                
                #----------waiting_time-----------
                if i3.find("span",attrs={"class":"DoctorCardstyle__Text-sc-uptab2-4 gaHgqr"}):
                    
                    waiting_time=i3.find("span",attrs={"class":"DoctorCardstyle__Text-sc-uptab2-4 gaHgqr"}).get_text()

                elif i3.find("span",attrs={"class":"DoctorCardstyle__Text-sc-uptab2-4 fzWsEs"}) :
                     waiting_time=i3.find("span",attrs={"class":"DoctorCardstyle__Text-sc-uptab2-4 fzWsEs"}).get_text()
                        
                
                elif i3.find("span",attrs={"class":"DoctorCardstyle__Text-sc-uptab2-4 gVQjRf"}):
                    waiting_time=i3.find("span",attrs={"class":"DoctorCardstyle__Text-sc-uptab2-4 gVQjRf"}).get_text()
                #print("waitng ",waiting_time)
               #---------------waiting_time-----------
            
            #-----------------------------التقييم
               
                #ss=i3.find("span",attrs={"class":"StarRatingstyle__StarRatingContainer-sc-16vjtpf-0 jjMOdP"}).get("data-testid")
               # print(ss)
            
               #---------الصوره
                if i3.find("div",attrs={"class":"DoctorCardSubComponentsstyle__DoctorImgContainer-sc-1vq3h7c-9 ccRniQ"}).find("img").get("src"):
                    photo=i3.find("div",attrs={"class":"DoctorCardSubComponentsstyle__DoctorImgContainer-sc-1vq3h7c-9 ccRniQ"}).find("img").get("src")
                elif i3.find("div",attrs={"class":"DoctorCardSubComponentsstyle__DoctorImgContainer-sc-1vq3h7c-9 ccRniQ"}).find("img").get("data-src"):
                    photo=i3.find("div",attrs={"class":"DoctorCardSubComponentsstyle__DoctorImgContainer-sc-1vq3h7c-9 ccRniQ"}).find("img").get("data-src")
                #print(photo)
                #-----------------------------الصوره------------
                
                #------------phone-----------------
                
                #phone=i3.find("span",attrs={"itemprop":"telephone"}).get_text()
                #print(phone)
                
                sql = "INSERT INTO doctor (id,id_general_specialty,photo,price,waiting_time,name,address) VALUES (%s, %s,%s,%s,%s,%s,%s)"

                
                val = (id_doctor,id_general_specialty, requests.get(photo).content,price,waiting_time,name,address)
                mycursor.execute(sql, val)

                connection.commit()
                id_con=id_con+1
                sql = "INSERT INTO doctor_subspecialtys (id,id_subspecialty,id_doctor) VALUES (%s, %s,%s)"
                val = (id_con,id_sub_apichalist,id_doctor)
                mycursor.execute(sql, val)

                connection.commit()
                
                
                
                
            soup2=change_url(url2,x)
            x=x+1
            if soup2.find("div",attrs={"class":"Paginationstyle__PaginationItem-sc-4txdoy-1 Paginationstyle__PaginationBox-sc-4txdoy-2 Paginationstyle__PaginationItemArrow-sc-4txdoy-4 Paginationstyle__PaginationItemNext-sc-4txdoy-5 kfQJUx"}):
               # print(x)
                #print("-----------------out------------")

                break
mycursor.close()    

def change_url(url2,x):
        #print(url2)
        d=soup2.find("a",attrs={"class":"Paginationstyle__TransparentA-sc-4txdoy-7 fKZvmm"}).get("href")
        #print("_----------------")
        if x==1:
            s=d        
            url3=url_original+s
            #print(url3)
            soup=BeautifulSoup(requests.get(url3).text,'html.parser')

        else:    

            url3=url_original+d
            #print(x)
            url3=url3.split("=")[0]+"="+str(x)
            
            #print(url3)
            soup=BeautifulSoup(requests.get(url3).text,'html.parser')

            
 
        return soup


    
    